In [5]:
import numpy as np
import pandas as pd
import xarray as xr
import pyproj
from rasterio.transform import Affine

import dask.dataframe as dd
import matplotlib.pyplot as plt
import matplotlib.colors as cl

import cartopy.crs as ccrs
import cartopy.feature as cfeature
import cartopy.geodesic as cgeo
crs = ccrs.PlateCarree()
import cmocean.cm as cm

from xgcm import Grid
from xhistogram.xarray import histogram
import warnings
warnings.filterwarnings("ignore")

import os
from glob import glob

import histlib.box as box
import histlib.aviso as aviso
import histlib.cstes as cstes
import histlib.diagnosis as diag
from histlib.cstes import labels, zarr_dir, matchup_dir

In [6]:
if True:
    from dask.distributed import Client
    from dask_jobqueue import PBSCluster
    #cluster = PBSCluster(cores=56, processes=28, walltime='04:00:00')
    #cluster = PBSCluster(cores=7, processes=7, walltime='04:00:00')
    cluster = PBSCluster(cores=5, processes=5, walltime='02:00:00')
    w = cluster.scale(jobs=4)
    #w = cluster.scale(jobs=3)
else:
    from dask.distributed import Client, LocalCluster
    cluster = LocalCluster()

client = Client(cluster)
client

Connection method: Cluster object,Cluster type: dask_jobqueue.PBSCluster
Dashboard: http://10.148.1.72:8787/status,
Dashboard: http://10.148.1.72:8787/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://10.148.1.72:47380,Workers: 0
Dashboard: http://10.148.1.72:8787/status,Total threads: 0
Started: Just now,Total memory: 0 B


# Create csv files

In [7]:
zarrs_cutoff = sorted(glob(os.path.join('/home/datawork-lops-osi/aponte/margot/historical_coloc_ok/cutoff_matchup', '*_2.zarr')))

In [8]:
zarrs = sorted(glob(os.path.join('/home/datawork-lops-osi/aponte/margot/historical_coloc_ok/matchup', '*')))

In [14]:
for i in range(len(zarrs)) :
    ds0 = xr.open_dataset(zarrs_cutoff[i])[['drifter_acc_x_25','drifter_coriolis_x_25']]
    ds1 = xr.open_dataset(zarrs[i])[['drogue_status','drifter_acc_x_0','drifter_coriolis_x_0','aviso_alti_ggx_adt', 'aviso_drifter_ggx_adt', 'es_cstrio_z15_drifter_wd_x', 'es_cstrio_z15_alti_wd_x']]
    path = os.path.join('/home/datawork-lops-osi/aponte/margot/historical_coloc_ok/coloc_err', zarrs[i].split('/')[-1].replace('zarr', 'csv'))
    df = xr.merge([ds0, ds1]).to_dataframe()
    df.where(df.drogue_status).dropna().to_csv(path)#only drogued drifters
    print(path)

ValueError: did not find a match in any of xarray's currently installed IO backends ['netcdf4', 'scipy', 'rasterio', 'zarr']. Consider explicitly selecting one of the installed engines via the ``engine`` parameter, or installing additional IO dependencies, see:
https://docs.xarray.dev/en/stable/getting-started-guide/installing.html
https://docs.xarray.dev/en/stable/user-guide/io.html

In [15]:
cluster.close()

In [10]:
zarrs = glob(os.path.join('/home/datawork-lops-osi/aponte/margot/historical_coloc_ok/coloc_err', '*'))

In [11]:
df = dd.concat([pd.read_csv(z) for z in zarrs])

In [12]:
df = df.compute()

In [13]:
len(df)

1896241